![](https://blogger.googleusercontent.com/img/a/AVvXsEjVBFVJg4z5YeCrBaFoTfJ7wcZxlFt9WrepKwOADEBtZe96pJI1KKryhjQupntMedLQgwOTdMTEHkyZm-LAHCrU7JD_1UNjoQdTGzvVe1-XVfA1rocSbCCmLfLqS7sm-_wKIsYwm5hW35RDF0wAhHYkL7MoWLM4QsowyQsNRAzb8xMaKhvgDVoMwH-b=s782)

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
import glob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from glob import glob
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image,display
import seaborn as sns
import matplotlib.image as mpimg
import scipy.spatial.distance as dist
from sklearn.model_selection import train_test_split
import os
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
train.head()

In [ ]:
# Check for Duplicates
train.duplicated().sum()

In [ ]:
print("Training data size",train.shape)
submission = pd.read_csv("../input/happy-whale-and-dolphin/sample_submission.csv")
submission.head()

In [ ]:
train['individual_id'].value_counts().hist()

In [ ]:
# missing data in training data 
total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending = False)
missing_train_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_data.head()

In [ ]:
# Occurance of individual_id in decreasing order(Top categories)
temp = pd.DataFrame(train.individual_id.value_counts().head(8))
temp.reset_index(inplace=True)
temp.columns = ['individual_id','count']
temp

In [ ]:
wd = pd.DataFrame(train.groupby(['individual_id'])['individual_id'].count())
wd.rename(columns={'individual_id': 'Count_Images'}, inplace=True)
wd.reset_index(inplace=True)
wd.sort_values(by=['Count_Images'],ascending=False, inplace=True)
wd['Cummulative_Count'] = wd['Count_Images'].cumsum()
wd['Cummulative_Pctg']= wd['Cummulative_Count']/wd['Count_Images'].sum()
wd['Row_id'] = np.arange(len(wd))
fig = plt.figure()
ax = plt.axes()
ax.plot(wd['Row_id'], wd['Cummulative_Pctg']);
ax.set(xlabel='Count of Classes', ylabel='Cummulative %',
       title='Cummulative distribution of images by class count');

In [ ]:
print("Number of classes that contribute 95% of total images is: "+ str(len(wd[wd['Cummulative_Pctg']<=0.95])))
print("Number of classes that contribute 99% of total images is: "+ str(len(wd[wd['Cummulative_Pctg']<=0.99])))
print(str(train['individual_id'].nunique()- len(wd[wd['Cummulative_Pctg']<=0.99]))+ " Classes contribute just about remaining 1% of images")
print("Number of classes with just two images is:"+ str(len(wd[wd['Count_Images']<=2])))

In [ ]:
# Plot the most frequent landmark_ids
plt.figure(figsize = (9, 8))
plt.title('Most frequent individual_id')
sns.set_color_codes("pastel")
sns.barplot(x="individual_id", y="count", data=temp,
            label="Count")
plt.show()

In [ ]:
# Occurance of landmark_id in increasing order
temp = pd.DataFrame(train.individual_id.value_counts().tail(8))
temp.reset_index(inplace=True)
temp.columns = ['individual_id','count']
temp

In [ ]:
# Plot the least frequent landmark_ids
plt.figure(figsize = (9, 8))
plt.title('Least frequent individual_id')
sns.set_color_codes("pastel")
sns.barplot(x="individual_id", y="count", data=temp,
            label="Count")
plt.show()

In [ ]:
train.nunique()

In [ ]:
from collections import Counter
a = train['individual_id'].tolist()
letter_counts = Counter(a)
df = pd.DataFrame.from_dict(letter_counts, orient='index')
df.plot(kind='bar')

In [ ]:
print("Number of classes under 20 occurences",(train['individual_id'].value_counts() <= 20).sum(),'out of total number of categories',len(train['individual_id'].unique()))

In [ ]:
sns.set()
plt.title('Training set: number of images per class(line plot)')
sns.set_color_codes("pastel")
individual_id = pd.DataFrame(train['individual_id'].value_counts())
individual_id.reset_index(inplace=True)
individual_id.columns = ['individual_id','count']
ax = individual_id['count'].plot(logy=True, grid=True)
locs, labels = plt.xticks()
plt.setp(labels, rotation=30)
ax.set(xlabel="individual_id", ylabel="Number of images")

In [ ]:
# Visualize outliers, min/max or quantiles of the landmarks count
sns.set()
ax = individual_id.boxplot(column='count')
ax.set_yscale('log')

In [ ]:
mainPath = '../input/happy-whale-and-dolphin/train_images/'
all_img_paths = [y for x in os.walk(mainPath) for y in glob(os.path.join(x[0], '*.jpg'))]
all_filenames = []
for filepath in all_img_paths:
    FileName = os.path.basename(filepath)
    all_filenames.append(FileName)
path_dict = dict(zip(all_filenames,all_img_paths))

In [ ]:
wd

In [ ]:
##Getting the list of ids where top 5 and bottom 5 categories
top5_cats = wd[wd['Row_id']<=4].individual_id.tolist()
bottom5_cats = wd[wd['Row_id']>=(wd['Row_id'].max()-4)].individual_id.tolist()

In [ ]:
print("Top 5 image categories are : " + str(top5_cats))
print("Bottom 5 image categories are : " + str(bottom5_cats))

In [ ]:
train['image']=train['individual_id']+str(".jpg")

In [ ]:
train

In [ ]:
img0 = cv2.imread('../input/happy-whale-and-dolphin/train_images/00021adfb725ed.jpg')

train_hist = plt.hist(img0.ravel(), bins = 256, color = 'orange', )
train_hist = plt.hist(img0[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
train_hist = plt.hist(img0[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
train_hist = plt.hist(img0[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
train_hist = plt.xlabel('Intensity Value')
train_hist = plt.ylabel('Count')
train_hist = plt.legend(['Total', 'Red Channel', 'Green Channel', 'Blue Channel'])
print('Intensity Histogram of Train Image')
plt.show()

In [ ]:
img1 = cv2.imread('../input/happy-whale-and-dolphin/test_images/000110707af0ba.jpg')

test_hist = plt.hist(img1.ravel(), bins = 256, color = 'orange', )
test_hist = plt.hist(img1[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
test_hist = plt.hist(img1[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
test_hist = plt.hist(img1[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
test_hist = plt.xlabel('Intensity Value')
test_hist = plt.ylabel('Count')
test_hist = plt.legend(['Total', 'Red Channel', 'Green Channel', 'Blue Channel'])
print('Intensity Histogram of Test Image')
plt.show()

In [ ]:
train_list = glob('../input/happy-whale-and-dolphin/train_images/*')

In [ ]:
plt.rcParams["axes.grid"] = False
f, axarr = plt.subplots(4, 3, figsize=(24, 22))

curr_row = 0
for i in range(12):
    example = cv2.imread(train_list[i])
    example = example[:,:,::-1]
    
    col = i%4
    axarr[col, curr_row].imshow(example)
    if col == 3:
        curr_row += 1